In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler, scale
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
import xgboost
from xgboost import XGBRegressor
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [2]:
X = pd.read_csv('./dataset-0510/train.csv', index_col='building_id')
X_test = pd.read_csv('./dataset-0510/test.csv', index_col='building_id')

y = X.total_price
X.drop(columns=['total_price'], inplace=True)

In [3]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.3, random_state=42) 

In [4]:
# step1. Imputation transformer for completing missing values.
step1 = ('Imputer', Imputer())
# step2. MinMaxScaler
step2 = ('MinMaxScaler', MinMaxScaler())
# step3. feature selection
#step3 = ('FeatureSelection', SelectFromModel(RandomForestRegressor()))
step3 = ('FeatureSelection', VarianceThreshold())

finally_step = ('model',
                XGBRegressor(
                    subsample=0.6,
                    reg_lambda=1e-5,
                    reg_alpha=0.01,
                    n_estimators=10000,
                    min_child_weight=6,
                    max_depth=9,
                    learning_rate=0.03,
                    gamma=0.03,
                    colsample_bytree=0.6,
                    tree_method='gpu_hist',
                    predictor='gpu_predictor',
                )
               )
parameters_for_testing = {
    'model__colsample_bytree':[0.4,0.6,0.8],
    'model__gamma':[0,0.03,0.1,0.3],
    'model__min_child_weight':[1.5,6,10],
    'model__learning_rate':[0.1,0.07, 0.03],
    'model__max_depth':[3,6,9],
    'model__n_estimators':[200, 500, 1000, 10000],
    'model__reg_alpha':[1e-5, 1e-2,  0.75],
    'model__reg_lambda':[1e-5, 1e-2, 0.45],
    'model__subsample':[0.6,0.95]
}

pipeline = Pipeline(steps=[step1, step2, finally_step])

random_search = RandomizedSearchCV(pipeline,
                                  parameters_for_testing)



/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [61]:
random_search.fit(X_train, y_train)
print('best params')
print (random_search.best_params_)

print('best score')
print (random_search.best_score_)



best params
{'model__subsample': 0.6, 'model__reg_lambda': 1e-05, 'model__reg_alpha': 0.01, 'model__n_estimators': 10000, 'model__min_child_weight': 6, 'model__max_depth': 9, 'model__learning_rate': 0.03, 'model__gamma': 0.03, 'model__colsample_bytree': 0.6}
best score
0.7662274417615335


In [5]:
pipeline.fit(X_train, y_train)

[23:32:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('Imputer',
                 Imputer(axis=0, copy=True, missing_values='NaN',
                         strategy='mean', verbose=0)),
                ('MinMaxScaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.6, gamma=0.03,
                              importance_type='gain', learning_rate=0.03,
                              max_delta_step=0, max_depth=9, min_child_weight=6,
                              missing=None, n_estimators=10000, n_jobs=1,
                              nthread=None, objective='reg:linear',
                              predictor='gpu_predictor', random_state=0,
                              reg_alpha=0.01, reg_lambda=1e-05,
                              scale_pos_weight=1, seed=None, silent=None,
               

## Train score

In [6]:
print(f"Train score: {pipeline.score(X_train, y_train)}")

Train score: 0.9999804658062686


In [7]:
y_pred = pipeline.predict(X_train)
y_pred

array([2.0832371e+06, 1.7793540e+07, 2.2721815e+06, ..., 3.1430259e+08,
       2.4359410e+06, 5.6971235e+06], dtype=float32)

In [8]:
#RMSLE = metrics.mean_squared_log_error(y_eval ,y_pred)
VARIANCE_SCORE = metrics.explained_variance_score(y_train, y_pred)
MSLE = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
r2 = metrics.r2_score(y_train, y_pred)

print("MSLE = ", MSLE)
#print("RMSLE = ", RMSLE)
print("VARIANCE_SCORE = ", VARIANCE_SCORE)
print("R2", r2)
'''
Stand score

Train score: 0.9573181748061892
MSLE =  11238714.006184377
VARIANCE_SCORE =  0.9573181796966825
R2 0.9573181748061892
'''

MSLE =  240432.33470813537
VARIANCE_SCORE =  0.999980465867024
R2 0.9999804658062686


'\nStand score\n\nTrain score: 0.9573181748061892\nMSLE =  11238714.006184377\nVARIANCE_SCORE =  0.9573181796966825\nR2 0.9573181748061892\n'

## Test score

In [66]:
print(f"Test score: {pipeline.score(X_eval, y_eval)}")

Test score: 0.5448241659946736


In [67]:
y_pred = pipeline.predict(X_eval)
y_pred

array([ 6443276. ,  6443276. , 13487648. , ...,  5495459.5, 43007330. ,
       13291290. ], dtype=float32)

In [68]:
#RMSLE = metrics.mean_squared_log_error(y_eval ,y_pred)
VARIANCE_SCORE = metrics.explained_variance_score(y_eval, y_pred)
MSLE = np.sqrt(metrics.mean_squared_error(y_eval, y_pred))
r2 = metrics.r2_score(y_eval, y_pred)

print("MSLE = ", MSLE)
#print("RMSLE = ", RMSLE)
print("VARIANCE_SCORE = ", VARIANCE_SCORE)
print("R2", r2)

'''
Stand score

Test score: 0.6710864790042561
MSLE =  32747921.6326721
VARIANCE_SCORE =  0.6711062838835371
R2 0.6710864790042561
'''

MSLE =  38524097.85315723
VARIANCE_SCORE =  0.5457591818164931
R2 0.5448241659946736


'\nStand score\n\nTest score: 0.6710864790042561\nMSLE =  32747921.6326721\nVARIANCE_SCORE =  0.6711062838835371\nR2 0.6710864790042561\n'

In [69]:
# save test predictions to file
predictions = pipeline.predict(X_test)
output = pd.DataFrame({'building_id': X_test.index, 'total_price': predictions})
output.to_csv('submission/XGB_predict.csv', index=False)

In [70]:
output

,building_id,total_price
0,X5gsdTWGS3W7JJQB,10885156.0
1,BTshNOJyKHnT2YIT,5244487.5
2,dhdymr0lV8N5kZOT,6018805.0
3,VEwyGGMcD56w5BOc,6380615.0
4,wmUeMoJZfsqaSX9b,4926371.0
5,EtBjGAHmHCe9t7TZ,5715631.5
6,hPNH34vmaZtvBtqc,7903760.0
7,wXjeI38bYDMJJwZC,12968612.0
8,fxZSGX6aPAFKU8W4,5309948.0
9,ewr0Fx6ign87OwaV,6380615.0
